![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
!pip install ibm_watsonx_ai
!pip install git+https://github.com/ibm-granite-community/utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.6 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.14.3-py3-none-any.whl size=77232 sha256=1970e4df5d106b5944b199c0b5bd69137f0ca34b14ac9a7fd4ab6140113c9206
  Stored in directory: /root/.cache/pip/wheels/cc/2f/6f/125918ad46d280d3bea58edf99f0757888ef6e7999db4b73b7
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.14.3-py3-none-any.whl size=662101 sha256=8eca00b1acf9c8dbc8917991505e39838b83cb76d396ff6d45443975cdcd34fc
  Stored in directory: /root/.cache/pip/wheels/f1/53/13/7c8fdeebdb847995d8ef349b4f695c595d8d31b30ae2a07ea2
  Created whe

In [2]:
import os
from ibm_watsonx_ai import APIClient, Credentials
from ibm_granite_community.notebook_utils import get_env_var
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=get_env_var("IBM_CLOUD_TOKEN")
)



IBM_CLOUD_TOKEN loaded from Google Colab secret.


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [3]:
model_id = "ibm/granite-3-3-8b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [4]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [5]:
project_id = get_env_var("PROJECT_ID")
space_id = get_env_var("SPACE_ID")


PROJECT_ID loaded from Google Colab secret.
SPACE_ID loaded from Google Colab secret.


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [6]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [7]:
from ibm_watsonx_ai.foundation_models.utils import Toolkit

vector_index_id = "e0cb43c3-093c-4481-8c97-cc3b0d6c076a"

def proximity_search( query ):

    api_client = APIClient(
        project_id=project_id,
        credentials=credentials,
    )

    document_search_tool = Toolkit(
        api_client=api_client
    ).get_tool("RAGQuery")

    config = {
    "vectorIndexId": vector_index_id,
    "projectId": project_id
    }

    results = document_search_tool.run(
        input=query,
        config=config
    )

    return results.get("output")

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [8]:
chat_messages = [];


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [9]:
question = input("Question: ")
grounding = proximity_search(question)
chat_messages.append({"role": "user", "content": f"""Use the following pieces of context to answer the question.

{grounding}

Question: {question}"""})
generated_response = model.chat(messages=chat_messages)
print(generated_response)


Question: Based on my transactional data, can you give me any recommendations on how to better budget and lower my expenses? I want to save more, so I need to cut down on unnecessary spending.
{'id': 'chatcmpl-481898daa643f2d2107273f9f52c6607---bbbe76d5-925c-485a-9291-ede292902902', 'object': 'chat.completion', 'model_id': 'ibm/granite-3-3-8b-instruct', 'model': 'ibm/granite-3-3-8b-instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Based on the provided transaction data, here are some recommendations to help you better budget and lower your expenses:\n\n1. **Categorize and Analyze Spending**: You already have transactions categorized into different areas like Investment, Health & Fitness, Rent, Salary, Travel, Food & Drink, Utilities, Shopping, and Other. This categorization is a great starting point. Analyze each category to understand where most of your money is going.\n\n2. **Identify Recurring Expenses**: Look for recurring expenses such as Rent, Salar

In [10]:
output = generated_response['choices'][0]['message']['content']

In [11]:
from IPython.display import Markdown, display

def print_markdown(string):
  display(Markdown(string))

print_markdown(output)

Based on the provided transaction data, here are some recommendations to help you better budget and lower your expenses:

1. **Categorize and Analyze Spending**: You already have transactions categorized into different areas like Investment, Health & Fitness, Rent, Salary, Travel, Food & Drink, Utilities, Shopping, and Other. This categorization is a great starting point. Analyze each category to understand where most of your money is going.

2. **Identify Recurring Expenses**: Look for recurring expenses such as Rent, Salary, Utilities, and Health & Fitness. These are typically fixed costs that are hard to avoid. However, you can still look for ways to minimize them. For example, consider negotiating your rent or finding cheaper utility options.

3. **Non-Essential Spending**: Identify non-essential spending categories like Travel, Food & Drink, Shopping, and Entertainment. These areas often have room for reduction. For instance, you have several entries in Food & Drink and Entertainment that could be reduced.

4. **Track Individual Expenses**: Within each category, look for individual transactions that stand out as unusually high. For example, in Health & Fitness, you have a $1,996.26 expense on 2022-01-21. Investigate if this is a recurring membership fee or a one-time, large purchase. If it's the latter, consider if there are cheaper alternatives.

5. **Limit Impulse Purchases**: Transactions like "My compare also argue" (Travel, $1,652.37) and "Best deal point" (Other, $4,975.00) suggest impulse buys or possibly large, one-time purchases. Try to limit such spending by waiting before making big purchases or setting a spending limit for smaller, discretionary items.

6. **Negotiate or Switch Service Providers**: For recurring expenses like Salary ($1,420.39 on 2022-01-04) and Utilities (ranging from $673.39 to $1,915.87), consider negotiating better rates or switching to more cost-effective providers.

7. **Set a Budget and Stick to It**: Create a monthly budget allocating funds to each category based on your needs and financial goals. Use budgeting apps or spreadsheets to track your spending against this budget. Adjust as necessary to ensure you're staying within your means.

8. **Build an Emergency Fund**: Having an emergency fund can prevent you from dipping into savings or using credit for unexpected expenses. Aim to save at least 3-6 months' worth of living expenses.

9. **Automate Savings**: Set up automatic transfers to a savings account each time you receive income. This ensures you save consistently without having to think about it.

10. **Review and Adjust Regularly**: Regularly review your spending habits and adjust your budget as needed. Life changes, and so do spending patterns, so it's essential to keep your budget relevant and effective.

By implementing these strategies, you should be able to identify areas where you can cut back on unnecessary spending and increase your savings.

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  